# 大きなプロジェクトが始まります!!

##製品プライサー

その説明から、何かがどれだけの費用がかかるかを推定できるモデル。

##データキュレーションパート1

今日は、データのサブセットであるHome Appliancesに焦点を当てることにより、スクラブとキュレーションを開始します。

データセットはこちらです：  
https://huggingface.co/datasets/mcauley-lab/amazon-reviews-2023

すべての製品データセットを備えたフォルダーはこちらです。  
https://huggingface.co/datasets/mcauley-lab/amazon-reviews-2023/tree/main/raw/meta_categories

In [ ]:
# 輸入

import os
from dotenv import load_dotenv
from huggingface_hub import login
from datasets import load_dataset, Dataset, DatasetDict
import matplotlib.pyplot as plt

In [ ]:
# 環境

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

In [ ]:
# Huggingfaceにログインします

hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

## もう1つのインポート - アイテムクラス

これを実行するときにメタの用語に同意する必要があるエラーが発生した場合は、提供するリンクに従って、指示に従ってください。数分以内にメタによって承認される必要があります。

メタが承認しない場合は、XX_MARKDOWN_LINK_XXの最後のセルを参照してください。

いずれかの問題 - 私にメッセージを送るか、私にメールしてください！  

HFにサインインした後にこの輸入品が来る必要があることを指摘してくれた学生ジョンS.に感謝します

In [ ]:
from items import Item

In [ ]:
%matplotlib inline

## データセットをロードします

次のセルでは、Huggingfaceからデータセットにロードします。

これにより、「trust_remote_codeがサポートされなくなった」などのエラーが発生した場合は、新しいセルでこのコマンドを実行してください： `！pipインストールデータセット== 3.6.0`を実行してからカーネルを再起動して、もう一度やり直してください。

In [ ]:
dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", f"raw_meta_Appliances", split="full", trust_remote_code=True)

In [ ]:
print(f"Number of Appliances: {len(dataset):,}")

In [ ]:
# 特定のデータポイントを調査します
datapoint = dataset[2]


In [ ]:
# 調査する

print(datapoint["title"])
print(datapoint["description"])
print(datapoint["features"])
print(datapoint["details"])
print(datapoint["price"])

In [ ]:
# 価格はいくつありますか？

prices = 0
for datapoint in dataset:
    try:
        price = float(datapoint["price"])
        if price > 0:
            prices += 1
    except ValueError as e:
        pass

print(f"There are {prices:,} with prices which is {prices/len(dataset)*100:,.1f}%")

In [ ]:
# 価格がある人のために、価格と長さを集めます

prices = []
lengths = []
for datapoint in dataset:
    try:
        price = float(datapoint["price"])
        if price > 0:
            prices.append(price)
            contents = datapoint["title"] + str(datapoint["description"]) + str(datapoint["features"]) + str(datapoint["details"])
            lengths.append(len(contents))
    except ValueError as e:
        pass

In [ ]:
# 長さの分布をプロットします

plt.figure(figsize=(15, 6))
plt.title(f"Lengths: Avg {sum(lengths)/len(lengths):,.0f} and highest {max(lengths):,}\n")
plt.xlabel('Length (chars)')
plt.ylabel('Count')
plt.hist(lengths, rwidth=0.7, color="lightblue", bins=range(0, 6000, 100))
plt.show()

In [ ]:
# 価格の分布をプロットします

plt.figure(figsize=(15, 6))
plt.title(f"Prices: Avg {sum(prices)/len(prices):,.2f} and highest {max(prices):,}\n")
plt.xlabel('Price ($)')
plt.ylabel('Count')
plt.hist(prices, rwidth=0.7, color="orange", bins=range(0, 1000, 10))
plt.show()

In [ ]:
# それで、このアイテムは何ですか？

for datapoint in dataset:
    try:
        price = float(datapoint["price"])
        if price > 21000:
            print(datapoint['title'])
    except ValueError as e:
        pass

これは私が見つけることができる最も近いものです - それは掘り出し物の価格で進んでいるように見えます!!

https://www.amazon.com/turbochef-electric-countertop-microwave-convection/dp/b01d05u9no/

##今度はデータセットをキュレートする時が来ました

1〜999米ドルの間のアイテムを選択します

アイテムインスタンスを作成します。このインスタンスは、適切なトークンザーを使用して180トークンに収まるようにテキストを切り捨てます

トレーニング中に使用するプロンプトを作成します。

十分な文字がない場合、アイテムは拒否されます。

##しかし、なぜ180トークン？

学生が私に素晴らしい質問をしました - なぜ私たちは180トークンに切り捨てられているのですか？その数をどのように判断しましたか？ （Moataz A.素晴らしい質問をありがとう）。

答え：これは「ハイパーパラメーター」の例です。言い換えれば、それは基本的に試行錯誤です！価格を測定するのに十分な有用な情報があるように、十分に多数のトークンが必要です。しかし、効率的にトレーニングできるように、数を低く抑えたいと考えています。これは7週目に動作していることがわかります。

私は、合理的と思われる数字から始め、180に落ち着く前にいくつかのバリエーションを実験しました。時間があれば、同じことをする必要があります！より良いバランスを見つけることで、私の結果を打ち負かすことができるかもしれません。この種の試行錯誤は少し不十分に聞こえるかもしれませんが、それはデータサイエンスの研究開発プロセスの重要な部分です。

トレーニングデータのトークンの数が少なくなる可能性がある別の興味深い理由があります。最終的に推論時にモデルを使用できるようになったら、新製品を提供し、価格を見積もらせたいと思います。そして、1-2文のような製品の短い説明を使用します。最高のパフォーマンスのために、推論時に提供する入力と同様にトレーニングデータをサイズする必要があります。

##しかし、私はitems.pyで表示されますそれは160トークンに入力を制約しますか？

Moataz A.からのもう1つの素晴らしい質問！製品の説明は、説明の前後にさらにテキストを追加してプロンプトに変えるため、160トークンに制限されています。これにより、合計で約180トークンになります。


In [ ]:
# 価格でそれぞれのアイテムオブジェクトを作成します

items = []
for datapoint in dataset:
    try:
        price = float(datapoint["price"])
        if price > 0:
            item = Item(datapoint, price)
            if item.include:
                items.append(item)
    except ValueError as e:
        pass

print(f"There are {len(items):,} items")

In [ ]:
# 最初のアイテムを見てください

items[1]

In [ ]:
# トレーニング中に使用されるプロンプトを調査する - モデルはこれを完了することを学びます

print(items[100].prompt)

In [ ]:
# テスト中に使用されるプロンプトを調査する - モデルはこれを完了する必要があります

print(items[100].test_prompt())

In [ ]:
# トークンカウントの分布をプロットします

tokens = [item.token_count for item in items]
plt.figure(figsize=(15, 6))
plt.title(f"Token counts: Avg {sum(tokens)/len(tokens):,.1f} and highest {max(tokens):,}\n")
plt.xlabel('Length (tokens)')
plt.ylabel('Count')
plt.hist(tokens, rwidth=0.7, color="green", bins=range(0, 300, 10))
plt.show()

In [ ]:
# 価格の分布をプロットします

prices = [item.price for item in items]
plt.figure(figsize=(15, 6))
plt.title(f"Prices: Avg {sum(prices)/len(prices):,.1f} and highest {max(prices):,}\n")
plt.xlabel('Price ($)')
plt.ylabel('Count')
plt.hist(prices, rwidth=0.7, color="purple", bins=range(0, 300, 10))
plt.show()

## サイドノート

Matplotlibがチャートで使用できるさまざまな色が気に入っている場合は、これをブックマークする必要があります。

https://matplotlib.org/stable/gallery/color/named_colors.html

あなたのための##Dodos：

 - アイテムのクラスを確認し、あなたがそれに満足していることを確認してください
 - いくつかのアイテムオブジェクトを調べ、トレーニングプロンプトを「item.prompt」で見て、 `item.test_prompt（）でプロンプトをテストします`
 - データをよりよく理解するために、さらにヒストグラムを作成します

##次回は他の多くの種類の製品と組み合わせます

エレクトロニクスや自動車のように。これにより、大規模なデータセットが得られ、トレーニングに最も適したサブセットを選択することにうるさくなります。